In [0]:
%pip install --upgrade "mlflow-skinny[databricks]>=2.4.1"
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 2.4.2
    Not uninstalling mlflow-skinny at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-63ecef1c-7438-43ce-83d3-2f9897262097
    Can't uninstall 'mlflow-skinny'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-store 0.13.6 requires pyspark<4,>=3.1.2, which is not installed.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow

mlflow.set_registry_uri("databricks-uc")

catalog_name = "databricks_llama_2_models" # Default catalog name when installing the model from Databricks Marketplace
version = 1


mlflow.pyfunc.get_model_dependencies(f"models:/{catalog_name}.models.llama_2_13b_chat_hf/{version}")

2023/11/15 11:29:46 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2023/11/15 11:30:19 INFO mlflow.utils.file_utils: Retrying download of chunk 7 for model/pytorch_model-00007-of-00028.bin
2023/11/15 11:30:35 INFO mlflow.utils.file_utils: Successfully downloaded chunk 7 for model/pytorch_model-00007-of-00028.bin
2023/11/15 11:31:34 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmp1lssr691/requirements.txt'.


'/tmp/tmp1lssr691/requirements.txt'

In [0]:
%pip install -r /tmp/tmp1lssr691/requirements.txt
dbutils.library.restartPython() 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.19.0
    Not uninstalling accelerate at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-63ecef1c-7438-43ce-83d3-2f9897262097
    Can't uninstall 'accelerate'. No files were found to uninstall.
  Attempting uninstall: transformers
    Found existing installation: transf

In [0]:
import mlflow

mlflow.set_registry_uri("databricks-uc")

catalog_name = "databricks_llama_2_models" # Default catalog name when installing the model from Databricks Marketplace
version = 1

# Create a Spark UDF to generate the response to a prompt
generate = mlflow.pyfunc.spark_udf(
    spark, f"models:/{catalog_name}.models.llama_2_13b_chat_hf/{version}", "string"
)

2023/11/15 11:32:56 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2023/11/15 11:33:36 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2023/11/15 11:33:37 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


In [0]:
import pandas as pd

# To have more than 1 input sequences in the same batch for inference, more GPU memory would be needed; swap to more powerful GPUs, or use Databricks Model Serving

df = spark.createDataFrame(
    pd.DataFrame(
        {
            "text": [
                "What is a large language model?",
                # "Write a short announcement of Llama 2 models in Databricks Marketplace.",
            ]
        }
    )
)
display(df)

generated_df = df.select(generate(df.text).alias("generated_text"))
display(generated_df)